In [19]:
from Utils import supplier_config as sc
import requests
import json
import pandas as pd

GET /v1/electricity-meter-points/{mpan}/meters/{serial_number}/consumption/    
GET /v1/gas-meter-points/{mprn}/meters/{serial_number}/consumption/  

In [2]:
s = requests.Session()

In [14]:
r = s.get(sc.API_ROOT + f'/v1/electricity-meter-points/{sc.electric_MPAN}/meters/{sc.electric_serial_number}/consumption/?page_size=25000',
    auth=(sc.API_KEY,'')
 )
r.raise_for_status()

In [21]:
elec_j = json.loads(r.content)

In [22]:
for res in elec_j['results']:
    print(res)
    break

{'consumption': 0.344, 'interval_start': '2023-03-11T23:30:00Z', 'interval_end': '2023-03-12T00:00:00Z'}


In [23]:
elec_df = (
    pd.DataFrame(elec_j['results'])
    .assign()

In [24]:
elec_df.sample()

,consumption,interval_start,interval_end
3986,0.283,2022-12-18T22:30:00Z,2022-12-18T23:00:00Z
